In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from collections import namedtuple
import jellyfish
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from mpire import WorkerPool
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils.extmath import safe_sparse_dot
import torch
from tqdm import tqdm
import wandb

from src.data.filesystem import fopen
from src.data.utils import load_train_test
from src.eval import metrics
from src.eval.utils import similars_to_ndarray
from src.models.autoencoder import get_best_autoencoder_matches
from src.models.utils import remove_padding

In [ ]:
# config

# from src.data.ancestry import load_train_test
# train_path = "../data/raw/records25k_data_train.csv"
# test_path = "../data/raw/records25k_data_test.csv"
# sample_all_names = True

given_surname = "given"
size = "freq"
test_train = "test"  # test on test or train
Config = namedtuple("Config", "train_path test_path anc_triplet_model_path fs_triplet_model_path sample_all_names")
config = Config(
    train_path=f"s3://familysearch-names/processed/tree-hr-{given_surname}-similar-train-{size}.csv.gz",
    test_path=f"s3://familysearch-names/processed/tree-hr-{given_surname}-similar-{test_train}-{size}.csv.gz",
    anc_triplet_model_path=f"s3://nama-data/data/models/anc-triplet-bilstm-100-512-40-05.pth",
    fs_triplet_model_path=f"s3://nama-data/data/models/fs-{given_surname}-{size}-triplet-bilstm-100-512-40-05.pth",
    sample_all_names=False,
)

In [ ]:
np.set_printoptions(suppress=True)
tqdm.pandas()
wandb.init(
    project="nama",
    entity="nama",
    name="60_compare_similarity",
    group=given_surname,
    notes="fs in-vocab",
    config=config._asdict(),
)

### Load data

In [ ]:
# read data

train, test = load_train_test([config.train_path, config.test_path])

input_names_train, weighted_actual_names_train, candidate_names_train = train
input_names_test, weighted_actual_names_test, candidate_names_test = test

In [ ]:
print("input_names_train", len(input_names_train))
print("input_names_test", len(input_names_test))
print("weighted_actual_names_train", len(weighted_actual_names_train))
print("weighted_actual_names_test", len(weighted_actual_names_test))
print("candidate_names_train", len(candidate_names_train))
print("candidate_names_test", len(candidate_names_test))

### Models

In [ ]:
# tfidf
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer="char_wb", min_df=10, max_df=0.5)
tfidf_X_train = tfidf_vectorizer.fit_transform(candidate_names_train)
tfidf_X_test = tfidf_vectorizer.transform(candidate_names_test)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# autoencoder with triplet loss
anc_triplet_model = torch.load(fopen(config.anc_triplet_model_path, "rb"), map_location=torch.device(device))
fs_triplet_model = torch.load(fopen(config.fs_triplet_model_path, "rb"), map_location=torch.device(device))

In [ ]:
SimilarityAlgo = namedtuple("SimilarityAlgo", "name min_threshold max_threshold distances")
similarity_algos = [
    SimilarityAlgo("fs_triplet", 0.01, 1.0, True),
    SimilarityAlgo("anc_triplet", 0.01, 1.0, True),
    SimilarityAlgo("tfidf", 0.5, 1.0, False),
    SimilarityAlgo("levenshtein", 0.5, 1.0, False),
    SimilarityAlgo("damerau_levenshtein", 0.5, 1.0, False),
    SimilarityAlgo("jaro_winkler", 0.5, 1.0, False),
]

In [ ]:
def calc_similarity_to(name, algo="levenshtein"):
    name = remove_padding(name)

    def calc_similarity(row):
        cand_name = remove_padding(row[0])
        similarity = 0.0
        if algo == "levenshtein":
            dist = jellyfish.levenshtein_distance(name, cand_name)
            similarity = 1 - (dist / max(len(name), len(cand_name)))
        elif algo == "damerau_levenshtein":
            dist = jellyfish.damerau_levenshtein_distance(name, cand_name)
            similarity = 1 - (dist / max(len(name), len(cand_name)))
        elif algo == "jaro_winkler":
            similarity = jellyfish.jaro_winkler_similarity(name, cand_name)

        return similarity

    return calc_similarity

#### Similarity Function

In [ ]:
def get_similars(shared, name=""):
    candidate_names_test, k, algo, tfidf_vectorizer, tfidf_X_test = shared
    if algo == "tfidf":
        x = tfidf_vectorizer.transform([name]).toarray()
        scores = safe_sparse_dot(tfidf_X_test, x.T).flatten()
    else:
        scores = np.apply_along_axis(calc_similarity_to(name, algo), 1, candidate_names_test[:, None])
    sorted_scores_idx = np.argsort(scores)[::-1][:k]
    candidate_names = candidate_names_test[sorted_scores_idx]
    candidate_scores = scores[sorted_scores_idx]

    return list(zip(candidate_names, candidate_scores))

#### Demo

In [ ]:
probe_name = "<bostelman>" if given_surname == "surname" else "<richard>"
get_similars((candidate_names_test, 10, "levenshtein", None, None), probe_name)

## Test tfidf

In [ ]:
probe_name = "<schumacher>" if given_surname == "surname" else "<richard>"
get_similars((candidate_names_test, 10, "tfidf", tfidf_vectorizer, tfidf_X_test), probe_name)

## Test levenshtein

In [ ]:
ix = 251
input_names_test[ix]

In [ ]:
weighted_actual_names_test[ix]

In [ ]:
k = 100  # Number of candidates to consider
similar_names_scores = [get_similars((candidate_names_test, k, "levenshtein", None, None), input_names_test[ix])]
similar_names_scores[0][:5]

In [ ]:
similar_names_scores = similars_to_ndarray(similar_names_scores)

In [ ]:
metrics.weighted_recall_at_threshold(weighted_actual_names_test[ix], similar_names_scores[0], 0.85)

In [ ]:
metrics.weighted_recall_at_threshold(weighted_actual_names_test[ix], similar_names_scores[0], 0.75)

# Evaluate each algorithm

In [ ]:
k = 1000  # Number of candidates to consider
n_pr_jobs = 16  # Number of jobs for precision_weighted_recall_at_thrshold

figure, ax = plt.subplots(1, 1, figsize=(20, 15))
ax.set_title("PR at threshold")
colors = cm.rainbow(np.linspace(0, 1, len(similarity_algos)))

for algo, color in zip(similarity_algos, colors):
    print(algo.name)
    if algo.name == "fs_triplet":
        similar_names_scores = get_best_autoencoder_matches(fs_triplet_model, input_names_test, candidate_names_test, k, 512)
    elif algo.name == "anc_triplet":
        similar_names_scores = get_best_autoencoder_matches(anc_triplet_model, input_names_test, candidate_names_test, k, 512)
    else:
        with WorkerPool(
            shared_objects=(candidate_names_test, k, algo.name, tfidf_vectorizer, tfidf_X_test)
        ) as pool:
            similar_names_scores = pool.map(get_similars, input_names_test, progress_bar=True)
        similar_names_scores = similars_to_ndarray(similar_names_scores)
    precisions, recalls = metrics.precision_weighted_recall_at_threshold(
        weighted_actual_names_test,
        similar_names_scores,
        min_threshold=algo.min_threshold,
        max_threshold=algo.max_threshold,
        distances=algo.distances,
        n_jobs=n_pr_jobs,
        progress_bar=True,
    )
    ax.plot(recalls, precisions, "o--", color=color, label=algo.name)

ax.legend()
plt.xlim([0, 1.0])
plt.ylim([0, 1.0])
plt.show()

In [ ]:
wandb.finish()